In [129]:
import sys, os, git, rstparse

In [130]:
current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/docs/inputs/shared-pyplot-files/")

In [131]:
tutorials = {"level0": ["lennard-jones-fluid"],
             "level1": ["breaking-a-carbon-nanotube"],
             "level2": ["polymer-in-water", "nanosheared-electrolyte"],
             "level3": ["water-adsorption-in-silica", "free-energy-calculation", "reactive-silicon-dioxide"]}

In [141]:
class RSTtoTEX:
    """Read RST file and convert it to TEX file."""
    def __init__(self, file_name, *args, **kwargs,):
        """Initialize"""
        super().__init__(*args, **kwargs)
        self.file_name = file_name

    def convert_file(self):
        """Main convert function."""
        self.read_rst()
        self.loop_through_file()

    def read_rst(self):
        """Convert the rst file into a list of strings"""
        rst = rstparse.Parser()
        with open(self.file_name) as f:
            rst.read(f)
        rst.parse()
        file_content = []
        for line in rst.lines:
            file_content.append(line)
        self.file_content = file_content

    def loop_through_file(self):
        """Loop over the rst file and extract the positions of items"""
        self.position_title = []
        self.position_labels = []
        self.position_hatnote = []
        for n, line in enumerate(self.file_content):
            self.detect_title_position(n, line)
            self.detect_label_positions(n, line)
            self.detect_hatnote_position(n, line)

    def detect_title_position(self, n, line):
        if line[:3] == "***":
            self.position_title.append(n-1)

    def detect_label_positions(self, n, line):
        if len(line.split()) == 2:
            if (line.split()[0] == '..') & (line.split()[1][0] == "_"):
                self.position_labels.append(n)

    def detect_hatnote_position(self, n, line):
        if len(line.split()) == 3:
            if (line.split()[0] == '..') & (line.split()[1][:-2] == "container") & (line.split()[2] == "hatnote"):
                self.position_hatnote.append(n)

In [142]:
for level in tutorials.keys():
    for tutorial in tutorials[level]:
        rst_file_name = git_path+'/docs/sphinx/source/tutorials/'+level+'/'+tutorial+'.rst'
        RST = RSTtoTEX(file_name = rst_file_name)
        RST.convert_file()

In [144]:
RST.position_hatnote

[5]

In [139]:
line.split()[2]

'hatnote'

In [36]:


def find_title(rst_file):
    title_line = []
    for n, line in enumerate(rst_file):
        if line[:3] == "***":
            title_line.append(n)
    assert len(title_line) == 1, """More than one title was found"""
    return rst_file[title_line[0]-1]

def find_label(rst_file):
    label_line = []
    for n, line in enumerate(rst_file):
        if line[:4] == ".. _":
            label_line.append(n)
    assert rst_file[label_line[0]][4:-1][-6:] == "-label", """Not a proper page label"""
    return rst_file[label_line[0]][4:-1]

def find_hatnote(rst_file):
    find_hatnote = []
    for n, line in enumerate(rst_file):
        if line == ".. container:: hatnote":
            find_hatnote.append(n)
    assert len(find_hatnote) == 1, """More than one hatline was found"""
    within_block = True
    n = find_hatnote[0]+1
    lines_in_block = []
    while within_block:
        line = rst_file[n]
        n += 1
        if line[:2] == '  ':
            within_block = True
            lines_in_block.append(line)
        elif line == '':
            pass
        else:
            within_block = False
    return lines_in_block[0][3:]

In [39]:

        rst_file = read_rst(rst_file_name)
        page_title = find_title(rst_file)
        page_label = find_label(rst_file)
        hatnote = find_hatnote(rst_file)

        print(hatnote)

The very basics of LAMMPS through a simple example
Breaking the bonds of a carbon nanotube under deformation
Stretching a small solvated molecule
Aqueous NaCl solution sheared by two walls
Dealing with a varying number of molecules
Simple sampling of a free energy barrier using umbrella sampling
Simulating a chemically reactive structure


In [40]:
find_justify = []
for n, line in enumerate(rst_file):
    if len(line.split(' ')) >= 4:
        if (line.split(' ')[0] == '..') & (line.split(' ')[2] =='container::') & (line.split(' ')[3] =='justify'):
            find_justify.append(n)
all_lines_in_block = []
for n in find_justify:
    within_block = True
    lines_in_block = []
    while within_block:
        n += 1
        line = rst_file[n]
        if line[:2] == '  ':
            within_block = True
            lines_in_block.append(line)
        elif line == '':
            lines_in_block.append('\n')
        else:
            within_block = False
    all_lines_in_block.append(lines_in_block)
# return lines_in_block[0][3:]

In [41]:
for block in all_lines_in_block:
    assert block[0] == '\n'
    assert block[-1] == '\n'
    block = block[1:-1]

    sentences = []
    sentence = ''
    for line in block:
        words = line.split()
        for word in words:
            if word == '\n':
                sentences.append(sentence)
                sentence = ''
            else:
                sentence += word + ' '
    sentences.append(sentence)

    print(sentences)

['The objective of this tutorial is to use a molecular dynamics system made of silicon dioxide (SiO2), and deform it until it breaks. The reactive force field *reaxff* is used, and a particular attention is given to the evolution of the charges of the atoms during the deformation of the structure. ']
['Create a folder, name it RelaxSilica/, and |download_silica_data| the initial topology of a small amorphous silica structure. The system was created by temperature annealing using another force field (|download_SiO.1990.vashishta|), therefore the structure is slightly different to what is expected from the reaxff force field. For instance, the average bond lengths, angles, and charges are likely to be different, and the structure needs to be relaxed again using reaxff. If you are interested, the input file used for creating the initial topology is available |lammps_input_creating|, but its description is not part of this tutorial. The Atoms section of the *silica.data* file starts like t

In [42]:
sentences

['Under ambient conditions, dandling oxygen are typically terminated by hydrogen atoms. Let us improve the current structure by decorating some of the dandling oxygen with hydrogen atoms, before relaxing it thanks to reaxff. Add hydrogen atoms to the dandling oxygens. Then relax the structure using reaxff with LAMMPS. Hydrogen atoms can be added using create_atoms command, gcmc, or external python script as I did here: ']